## **PROYECTO 1**
#### **Limpieza de Datos**

Ignacio Méndez Alvarez (22613), Diego Soto Flores (22737) y Pablo Herrera Barbales (21227)

Enlace repositorio de GitHub: https://github.com/ignaciomendeza/PROYECTO1_Data_Science.git

### Carga de librerias

In [1]:
import pandas as pd
import glob
import re
import unicodedata
from rapidfuzz import fuzz

### Carga de archivos y visualización

In [5]:
# Se carga y combinan múltiples archivos CSV desde una carpeta específicasaltando filas de encabezado no necesarias y líneas de pie de página.

ruta = "mineduc/*.csv"

# Leer y procesar cada archivo CSV encontrado en la ruta:
dfs = [
    pd.read_csv(archivo, skiprows=25, skipfooter=5, engine="python")
    for archivo in sorted(glob.glob(ruta))
]

df_final = pd.concat(dfs, ignore_index=True)
print(df_final.shape)


(6600, 17)


In [6]:
# Verificación de columnas
df_final.head()
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6600 entries, 0 to 6599
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODIGO           6600 non-null   object
 1   DISTRITO         6600 non-null   object
 2   DEPARTAMENTO     6600 non-null   object
 3   MUNICIPIO        6600 non-null   object
 4   ESTABLECIMIENTO  6600 non-null   object
 5   DIRECCION        6598 non-null   object
 6   TELEFONO         6554 non-null   object
 7   SUPERVISOR       6600 non-null   object
 8   DIRECTOR         6574 non-null   object
 9   NIVEL            6600 non-null   object
 10  SECTOR           6600 non-null   object
 11  AREA             6600 non-null   object
 12  STATUS           6600 non-null   object
 13  MODALIDAD        6600 non-null   object
 14  JORNADA          6600 non-null   object
 15  PLAN             6600 non-null   object
 16  DEPARTAMENTAL    6600 non-null   object
dtypes: object(17)
memory usage: 876.7

### Formato CODIGO

In [7]:
# Verificación del formato de la columna 'CODIGO' en el DataFrame `df_final`.
pattern = re.compile(r"^\d{2}-\d{2}-\d{4}-\d{2}$")
filas_incorrectas = df_final[~df_final["CODIGO"].astype(str).apply(lambda x: bool(pattern.match(x)))]
print(filas_incorrectas)


Empty DataFrame
Columns: [CODIGO, DISTRITO, DEPARTAMENTO, MUNICIPIO, ESTABLECIMIENTO, DIRECCION, TELEFONO, SUPERVISOR, DIRECTOR, NIVEL, SECTOR, AREA, STATUS, MODALIDAD, JORNADA, PLAN, DEPARTAMENTAL]
Index: []


Todas las filas siguen el formato establecido.

### Formato DISTRITO 

In [8]:
# Verificación del formato de la columna 'DISTRITO' en el DataFrame `df_final`.
pattern = re.compile(r"^\d{2}-\d{3}$")
filas_incorrectas = df_final[~df_final["DISTRITO"].astype(str).apply(lambda x: bool(pattern.match(x)))]
print(filas_incorrectas)


Empty DataFrame
Columns: [CODIGO, DISTRITO, DEPARTAMENTO, MUNICIPIO, ESTABLECIMIENTO, DIRECCION, TELEFONO, SUPERVISOR, DIRECTOR, NIVEL, SECTOR, AREA, STATUS, MODALIDAD, JORNADA, PLAN, DEPARTAMENTAL]
Index: []


Todas las filas siguen el formato establecido.

### Formato ESTABLECIMIENTO

In [9]:
# Verificación de mayúsculas en la columna 'ESTABLECIMIENTO' del DataFrame `df_final`.
filas_incorrectas = df_final[df_final["ESTABLECIMIENTO"] != df_final["ESTABLECIMIENTO"].str.upper()]
print(filas_incorrectas)



Empty DataFrame
Columns: [CODIGO, DISTRITO, DEPARTAMENTO, MUNICIPIO, ESTABLECIMIENTO, DIRECCION, TELEFONO, SUPERVISOR, DIRECTOR, NIVEL, SECTOR, AREA, STATUS, MODALIDAD, JORNADA, PLAN, DEPARTAMENTAL]
Index: []


Todas las filas siguen el formato establecido.

In [10]:
# Detección de valores repetidos en la columna 'ESTABLECIMIENTO' del DataFrame `df_final`.
repetidos = df_final[df_final["ESTABLECIMIENTO"].duplicated(keep=False)]
print(repetidos.sort_values("ESTABLECIMIENTO"))


             CODIGO DISTRITO   DEPARTAMENTO      MUNICIPIO  \
4292  17-05-0211-46   17-007          PETEN    LA LIBERTAD   
4321  17-05-0335-46   17-007          PETEN    LA LIBERTAD   
4315  17-05-0319-46   17-007          PETEN    LA LIBERTAD   
4304  17-05-0260-46   17-007          PETEN    LA LIBERTAD   
5947  06-01-0118-46   06-039     SANTA ROSA        CUILAPA   
...             ...      ...            ...            ...   
520   04-01-3068-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
392   04-01-0058-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
519   04-01-3067-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
473   04-01-0357-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
431   04-01-0230-46   04-001  CHIMALTENANGO  CHIMALTENANGO   

                                        ESTABLECIMIENTO  \
4292          "CENTRO DE EDUCACION INTEGRAL EL NARANJO"   
4321          "CENTRO DE EDUCACION INTEGRAL EL NARANJO"   
4315          "CENTRO DE EDUCACION INTEGRAL EL NARANJO"   
4304          "CENT

In [11]:
# Detección de nombres de establecimientos con alta similitud (mayor al 95%) utilizando la librería RapidFuzz.
nombres = df_final["ESTABLECIMIENTO"].unique()
resultados = []
for i, nombre in enumerate(nombres):
    for j in range(i + 1, len(nombres)):  
        similitud = fuzz.ratio(nombre, nombres[j])  
        if similitud > 95 and similitud < 100:      
            resultados.append({
                "Nombre_Original": nombre,
                "Nombre_Comparado": nombres[j],
                "Similitud (%)": similitud
            })

df_parecidos = pd.DataFrame(resultados).sort_values("Similitud (%)", ascending=False)
print(f"Total de pares con similitud > 95%: {len(df_parecidos)}")
df_parecidos


Total de pares con similitud > 95%: 883


,Nombre_Original,Nombre_Comparado,Similitud (%)
296,"COLEGIO ""CENTRO DE FORMACION PROFESIONAL PRE-U...","COLEGIO ""CENTRO DE FORMACION PROFESIONAL PREUN...",99.393939
529,COLEGIO EDUCATIVO PRIVADO MIXTO DE FORMACION I...,COLEGIO EDUCATIVO PRIVADO MIXTO DE FORMACION I...,99.363057
215,INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA ...,INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA ...,99.328859
329,INSTITUTO MIXTO MUNICIPAL DE EDUCACIÓN BÁSICA ...,INSTITUTO MIXTO MUNICIPAL DE EDUCACIÓN BÁSICA ...,99.310345
87,CENTRO DE ESTUDIOS TECNICOS Y AVANZADOS DE CHI...,CENTRO DE ESTUDIOS TECNICOS Y AVANZADOS DE CHI...,99.290780
...,...,...,...
58,PROGRAMA NACIONAL DE EDUCACIÓN ALTERNATIVA -PR...,"PROGRAMA NACIONAL DE EDUCACION ALTERNATIVA, PR...",95.049505
662,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA ...,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADO-...,95.049505
346,"PROGRAMA NACIONAL DE EDUCACIÓN ALTERNATIVA , P...",PROGRAMA NACIONA DE EDUCACIÓN ALTERNATIVA -PRO...,95.049505
745,PROGRAMA NACIONA DE EDUCACIÓN ALTERNATIVA -PRO...,"PROGRAMA NACIONAL DE EDUCACIÓN ALTERNATIVA, PR...",95.049505


In [12]:
# Limpia y normaliza los valores de una columna de nombres de establecimientos educativos.
def limpiar_establecimiento(df, columna="ESTABLECIMIENTO"):
    
    def normalizar_tildes(texto):
        return unicodedata.normalize("NFC", texto)


    correcciones_seguras = {
        "INSDUSTRIAL": "INDUSTRIAL",
        "BILINGUE": "BILINGÜE"
    }

    correcciones_tildes = {
        "BILINGÚE": "BILINGÜE",
        "BINLINGÜE": "BILINGÜE",
        "TECNOLGÍA": "TECNOLOGÍA",
        "TECNOLOGIA": "TECNOLOGÍA",
        "TECNÓLOGICO": "TECNOLÓGICO",
        "EDCACIÓN": "EDUCACIÓN",
        "JÉSUS": "JESÚS",
        "EVANGELICO": "EVANGÉLICO",
        "COMPUTACION": "COMPUTACIÓN",
        "INFORMATICA": "INFORMÁTICA",
        "TECNICAS": "TÉCNICAS",
        "FORMACION": "FORMACIÓN",
        "BASICA": "BÁSICA"
    }


    df[columna] = (
        df[columna]
        .astype(str)                                
        .str.upper()                                
        .apply(normalizar_tildes)                   
        .str.replace(r"[-_]", " ", regex=True)      
        .str.replace(r"\s+", " ", regex=True)       
        .str.strip()                                
        .str.rstrip(".")                                   
        .str.replace(r"\(\s*(.*?)\s*\)", r"(\1)", regex=True)  
        .replace(correcciones_seguras, regex=True)  
        .replace(correcciones_tildes, regex=True)   
        .str.replace(r"(\d),(\d{3})", r"\1\2", regex=True)      
        .str.replace(r"\bNO\.?\s*(\d+)", r"NO. \1", regex=True) 
        .str.replace(r"(NO\. \d+),", r"\1, ", regex=True)       
        .str.replace(r"\s+,", ",", regex=True)      
        .str.replace(r",\s*", ", ", regex=True)     
        .str.replace(r"\s+", " ", regex=True)       
        .str.strip()
    )

    return df

df_final = limpiar_establecimiento(df_final, "ESTABLECIMIENTO")

In [13]:
# Coloca las comillas dobles, respetando los casos donde se usan comillas simples
df_final["ESTABLECIMIENTO"] = (
    df_final["ESTABLECIMIENTO"]
    .astype(str)  # Asegurar que todos los valores sean strings
    .str.replace(r"(?<!\w)'|'(?!\w)", '"', regex=True)  
)

# Dirección

In [14]:
def limpiar_direccion(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    
    s = unicodedata.normalize("NFC", s).upper()
    
    # Unificar variantes de AVENIDA
    s = re.sub(r"\b(AVENIDA|AVENIDA\.|AVE|AVE\.|AV|AV\.)\b", "AVENIDA", s)
    
    # Normalizar ordinales femeninos
    s = re.sub(r"\b(\d+)\s*A\b", r"\1ª", s)
    s = re.sub(r"\b(\d+)\s*RA\.?\b", r"\1ª", s)
    s = re.sub(r"\b(\d+)\s*DA\.?\b", r"\1ª", s)
    s = re.sub(r"\b(\d+)\s*ERA\.?\b", r"\1ª", s)
    s = re.sub(r"\b(\d+)\s*TA\.?\b", r"\1ª", s)
    s = re.sub(r"\b(\d+)\s*A\b", r"\1ª", s)
    
    # Números antes de AVENIDA 
    s = re.sub(r"\b([1-9]\d*)\s*AVENIDA\b", r"\1ª. AVENIDA", s)

    # Cambiar valores antes de AVENIDA
    s = re.sub(r"\b([1-9]\d*)AV\.?\b", r"\1ª. AVENIDA", s)

    # KM y variantes
    s = re.sub(r"\b(KM|KMS?|KIL|KIL\.|KILOMETRO|KILÓMETRO)\.?\s*(\d+)\b", r"KM. \2", s)
    
    # Unificar variantes de CALLE
    s = re.sub(r"\b(CALE|CALLE\.?|C\.|CLLE|CL)\b", "CALLE", s)
    
    # Cambiar 'O' solita por '0'
    s = re.sub(r"\bO\b", "0", s)

    s = re.sub(r"\b0ª\. AVENIDA\b", "0 AVENIDA", s)
        
    s = re.sub(r"\b([A-Z])\1+\b", r"\1", s)
    
    s = re.sub(r"\bDIAGNONAL\b", "DIAGONAL", s)

    # Normalizar espacios múltiples
    s = re.sub(r"\s+", " ", s).strip()
    
    # Asegurar espacio después de AVENIDA
    s = re.sub(r"\bAVENIDA\b(?!\s)", "AVENIDA ", s)
        
    s = re.sub(r'\bCINCO\s+GUION\s+DOCE\b', '', s, flags=re.IGNORECASE)
    s = re.sub(r'\s+', ' ', s).strip() 

    return s

if "DIRECCION" in df_final.columns:
    df_final["DIRECCION"] = df_final["DIRECCION"].apply(limpiar_direccion)


In [15]:
# Se coloca N/A en celdas vacías
df_final["DIRECCION"] = df_final["DIRECCION"].apply(
    lambda x: "N/A" if pd.isna(x) or str(x).strip() == "" else x
)

# Teléfono

In [16]:
def limpiar_telefono(s: str) -> str:
    s = "" if pd.isna(s) else str(s).strip()
    
    # Elimina todo lo que no sea dígito
    numeros = re.findall(r"\d+", s)
    
    if not numeros:
        return "N/A"
    
    tel = numeros[0]
    
    # Si no tiene exactamente 8 dígitos, lo marca como N/A
    if len(tel) != 8:
        return "N/A"
    
    # Inserta guion después de los primeros 4 dígitos
    tel_formateado = f"{tel[:4]}-{tel[4:]}"
    
    return tel_formateado

# Aplicarlo a df_final
if "TELEFONO" in df_final.columns:
    df_final["TELEFONO"] = df_final["TELEFONO"].apply(limpiar_telefono)


# Supervisor/director

In [17]:
def normalize_nfc(s: str) -> str:
    return unicodedata.normalize("NFC", s)

def strip_diacritics(s: str) -> str:
    s = unicodedata.normalize("NFD", s)
    s = "".join(c for c in s if unicodedata.category(c) != "Mn")
    return unicodedata.normalize("NFC", s)

def limpiar_nombre_persona(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    s = normalize_nfc(s).upper().strip()
    s = re.sub(r"[“”]", '"', s)
    s = re.sub(r"[‘’]", "'", s)
    s = re.sub(r'^"+|"+$', "", s)           
    s = re.sub(r"[^\w\s'.-]", " ", s)       
    s = re.sub(r"\s+", " ", s).strip()
    return s

def clave_bloqueo(s: str) -> str:
    s2 = strip_diacritics(s)
    s2 = re.sub(r"[^A-Z\s]", " ", s2.upper())
    toks = sorted(set(s2.split()))
    return " ".join(toks)

def construir_mapeo_canon(df_col_std: pd.Series, df_col_block: pd.Series, umbral=95):
    mapping = {}
    tmp = pd.DataFrame({"STD": df_col_std, "BLOCK": df_col_block}).dropna().drop_duplicates()
    #Devuelve dict: nombre_std -> nombre_canónico
    for block, grp in tmp.groupby("BLOCK"):
        vals = grp["STD"].tolist()
        if len(vals) == 1:
            mapping[vals[0]] = vals[0]
            continue
        clusters = []
        for v in vals:
            asignado = False
            for cl in clusters:
                if any(fuzz.token_set_ratio(v, w) >= umbral for w in cl):
                    cl.append(v)
                    asignado = True
                    break
            if not asignado:
                clusters.append([v])
        freqs = df_col_std.value_counts()
        for cl in clusters:
            canon = max(cl, key=lambda x: freqs.get(x, 0))
            for v in cl:
                mapping[v] = canon
    return mapping

# Aplicar limpieza a SUPERVISOR / DIRECTOR
for col in ["SUPERVISOR", "DIRECTOR"]:
    if col in df_final.columns:
        df_final[col + "_STD"] = df_final[col].apply(limpiar_nombre_persona)
        df_final[col + "_BLOCK"] = df_final[col + "_STD"].apply(clave_bloqueo)
        
        mapeo = construir_mapeo_canon(df_final[col + "_STD"], df_final[col + "_BLOCK"], umbral=95)
        df_final[col + "_CANON"] = df_final[col + "_STD"].map(mapeo)
        
        mask = df_final[col + "_STD"] != df_final[col + "_CANON"]
        df_final.loc[mask, col] = df_final.loc[mask, col + "_CANON"]
        
        df_final[col] = df_final[col].apply(lambda x: "N/A" if not re.search(r"[A-Z]", str(x).upper()) else x)

        df_final.drop(columns=[col + "_STD", col + "_BLOCK", col + "_CANON"], inplace=True)


# Plan

In [18]:
def normalize_nfc(s: str) -> str:
    return unicodedata.normalize("NFC", s)

# Limpieza base para comparar 
def limpiar_plan_std(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    s = normalize_nfc(s).upper().strip()
    s = re.sub(r"[“”]", '"', s)
    s = re.sub(r"[‘’]", "'", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Clave de bloqueo para agrupar variantes
def bloque_plan(s: str) -> str:
    s2 = unicodedata.normalize("NFD", s)
    s2 = "".join(c for c in s2 if unicodedata.category(c) != "Mn")  # sin tildes
    s2 = re.sub(r"[^A-Z0-9\s]", " ", s2.upper())  # elimina todo lo no alfanumérico ((), -, etc.)
    toks = sorted(set(s2.split()))
    return " ".join(toks)

def construir_mapeo_canon(df_col_std: pd.Series, df_col_block: pd.Series, umbral=90):
    
    mapping = {}
    tmp = pd.DataFrame({"STD": df_col_std, "BLOCK": df_col_block}).dropna().drop_duplicates()
    freqs = df_col_std.value_counts()

    for block, grp in tmp.groupby("BLOCK"):
        vals = grp["STD"].tolist()
        if len(vals) == 1:
            mapping[vals[0]] = vals[0]
            continue

        clusters = []
        for v in vals:
            asignado = False
            for cl in clusters:
                if any(fuzz.token_set_ratio(v, w) >= umbral for w in cl):
                    cl.append(v)
                    asignado = True
                    break
            if not asignado:
                clusters.append([v])

        for cl in clusters:
            canon = max(cl, key=lambda x: freqs.get(x, 0))
            for v in cl:
                mapping[v] = canon

    return mapping

# Aplicamos a Plan
if "PLAN" in df_final.columns:
    df_final["PLAN_STD"]   = df_final["PLAN"].apply(limpiar_plan_std)
    df_final["PLAN_BLOCK"] = df_final["PLAN_STD"].apply(bloque_plan)

    mapeo = construir_mapeo_canon(df_final["PLAN_STD"], df_final["PLAN_BLOCK"], umbral=90)
    df_final["PLAN_CANON"] = df_final["PLAN_STD"].map(mapeo).fillna(df_final["PLAN_STD"])

    # Sobrescribir la columna original con el canónico
    df_final["PLAN"] = df_final["PLAN_CANON"]

    df_final["PLAN"] = (
        df_final["PLAN"]
        .str.replace(r"(?<!\s)\(", " (", regex=True)  
        .str.replace(r"\s+", " ", regex=True)         
        .str.strip()
    )

    df_final.drop(columns=["PLAN_STD", "PLAN_BLOCK", "PLAN_CANON"], inplace=True, errors="ignore")

#### **DATA LIMPIA**

In [19]:
df_final.to_excel("df_final.xlsx", index=False, engine="openpyxl")

In [20]:
df_final.to_csv("df_final.csv", index=False, sep=",", encoding="utf-8")